# The Querying Pipeline

## How Postgres Handles Queries Step by Step:

```sql
SELECT *
FROM users
WHERE username = 'Alyson14';
```
1. **Parser**
   - The query is first sent to the **parser**, where it is broken down into a structured format called a **parse tree**.  
   - The parser checks for **syntax errors** and validates the SQL query to ensure that it follows the SQL standard.  
   - For example, it ensures that `SELECT`, `FROM`, and `WHERE` are valid SQL keywords, and that `users` is a valid table name in the database schema.

2. **Rewrite**
   - The **query rewriter** takes the parsed query and applies any applicable **rules** or **transformations**.  
   - This step involves applying **views**, **rules**, or **triggers** that may modify the query structure.  
   - For example, if the query references a view instead of a table, the rewriter replaces the view with its underlying query.

3. **Planner**
   - The **query planner** (or **optimizer**) generates an **execution plan**.  
   - Postgres considers multiple strategies (e.g., **sequential scan**, **index scan**) to determine the most **cost-efficient method** of executing the query.  
   - The planner uses the database's **statistics** and **indexes** to estimate costs, such as the number of disk I/O operations and CPU resources required.  
   - For instance, if there’s an index on `username`, the planner may choose an **index scan** over a **sequential scan** to optimize performance.

4. **Execute**
   - The **executor** runs the query based on the chosen plan from the planner.  
   - Data is fetched row by row (or in batches) according to the plan's strategy.  
   - For example, if an **index scan** is chosen, Postgres uses the index to locate the specific row(s) matching `username = 'Alyson14'`.  
   - The results are returned to the client in the format specified by the query (e.g., `SELECT *` returns all columns for the matching row).

---

### When to Use Indexes
- Use indexes for columns that are frequently queried in **WHERE**, **JOIN**, or **ORDER BY** clauses.  
- Avoid adding indexes to columns with a high number of **duplicate values** (low cardinality) unless needed for specific use cases.  
- Monitor query performance with `EXPLAIN` or `EXPLAIN ANALYZE` to identify slow queries that may benefit from indexes.  
- Remember that indexes speed up **read operations** but may slow down **write operations** (INSERT, UPDATE, DELETE).

---

### Types of Indexes in Postgres
1. **B-Tree Indexes**  
   - Default index type, ideal for equality and range queries.  
2. **Hash Indexes**  
   - Optimized for equality comparisons, such as `=` and `!=`.  
3. **GIN (Generalized Inverted Index)**  
   - Suitable for full-text search and JSONB fields.  
4. **GiST (Generalized Search Tree)**  
   - Useful for complex queries, such as geometric data.  
5. **BRIN (Block Range Index)**  
   - Ideal for large, sequentially ordered datasets.



# Query Cost Formula

The cost of a query in PostgreSQL is calculated using the following formula:

Cost = (# pages read sequentially) * 1.0  
     + (# pages read at random) * 4.0  
     + (# rows scanned) * 0.01  
     + (# index entries scanned) * 0.005  
     + (# times function/operator evaluated) * 0.0025


### Breakdown of Components:
- **Sequential Pages**:  
  Pages read sequentially cost `1.0` per page (cheaper than random access).  

- **Random Pages**:  
  Pages read randomly cost `4.0` per page (more expensive than sequential access).  

- **Rows Scanned**:  
  Each row scanned adds `0.01` to the total cost.  

- **Index Entries Scanned**:  
  Each scanned index entry adds `0.005` to the total cost.  

- **Function/Operator Evaluation**:  
  Each function/operator evaluation adds `0.0025` to the total cost.

### Example:
If a query:
- Reads 10 sequential pages,
- Reads 2 random pages,
- Scans 500 rows,
- Scans 200 index entries, and
- Evaluates functions/operators 100 times,

The total cost would be:

Cost = (10 * 1.0) + (2 * 4.0) + (500 * 0.01) + (200 * 0.005) + (100 * 0.0025)  
     = 10 + 8 + 5 + 1 + 0.25  
     = 24.25

